In [4]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
import toml
from src import settings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.utils import fileio

CONFIG_PATH = os.path.join(settings.CONFIG_DIR, "main.toml")
with open(CONFIG_PATH, "r") as file:
    config = toml.load(file)
TIME_WINDOW=config["TIME_WINDOW"]
INPUT_PATH = os.path.join(settings.RESULTS_DIR, "local_measures_snapshots", f"{TIME_WINDOW}_sec_window/")
all_treatments = fileio.load_multiple_folders(INPUT_PATH)
all_treatments = {key: value for key, value in all_treatments.items() if key in config["TREATMENTS"]}

dataframes = []
for treatment_name, treatment_path in all_treatments.items():
    all_groups = fileio.load_multiple_folders(treatment_path)
    treatment_dataframes = []
    for group_name, group_path in all_groups.items():
        all_snapshots = fileio.load_files_from_folder(group_path)
        group_dataframes = []  # Create a list to store DataFrames for each group
        for snapshot_name, snapshot_path in all_snapshots.items():
            df = pd.read_csv(snapshot_path, index_col=0) 
            df['Snapshot'] = snapshot_name.replace('.csv', '')
            df['Group'] = group_name
            group_dataframes.append(df)  # Append the DataFrame to the list for the group
        group_combined_df = pd.concat(group_dataframes)  # Combine DataFrames for the group
        treatment_dataframes.append(group_combined_df)  # Append the combined DataFrame to the list

    combined_df = pd.concat(treatment_dataframes)
    combined_df = combined_df.set_index(['Group'], append=True)
    combined_df.index.names = ['Fly', 'Group']
    combined_df = combined_df.reorder_levels(['Fly', 'Group'])
    combined_df['Treatment'] = treatment_name
    combined_df = combined_df.set_index('Treatment', append=True)
    dataframes.append(combined_df)

combined_data = pd.concat(dataframes)

combined_data_reset = combined_data.reset_index()
combined_data_reset['Snapshot'] = combined_data_reset['Snapshot'].astype(int)
FONT_SIZE = 12
num_cols = 3
num_rows = len(combined_data.columns)

# Create a new main figure to hold the main title
fig_main = plt.figure(figsize=(52, 6 * num_rows))
# fig_main.suptitle(f"Distribution of local measures for time window of {TIME_WINDOW} seconds", fontsize=30)

grid = plt.GridSpec(num_rows, num_cols, hspace=0.4, width_ratios=[16, 6, 6])
for i, measure_name in enumerate(combined_data.columns.tolist()):
    if measure_name == "Snapshot":
        continue
    
    ax1 = plt.subplot(grid[i, 0])
    ax2 = plt.subplot(grid[i, 1])
    ax3 = plt.subplot(grid[i, 2])

    sns.boxplot(data=combined_data_reset, x="Snapshot", y=measure_name, hue="Treatment", ax=ax1)
    ax1.set_xlabel("Snapshot", fontsize=FONT_SIZE)
    ax1.set_ylabel(measure_name, fontsize=FONT_SIZE)
    ax1.set_title(f'Distribution of {measure_name}', fontsize=FONT_SIZE, loc='center')

    sns.boxplot(data=combined_data_reset, x="Treatment", y=measure_name, ax=ax2)
    ax2.set_xlabel("Treatment", fontsize=FONT_SIZE)
    ax2.set_ylabel(measure_name, fontsize=FONT_SIZE)
    ax2.set_title(f'Distribution of {measure_name}', fontsize=FONT_SIZE, loc='center')

    sns.histplot(data=combined_data_reset, x=measure_name, hue="Treatment",
                 stat="probability", common_norm=False, element="step", kde=True, ax=ax3)
    ax3.set_xlabel(measure_name, fontsize=FONT_SIZE)
    ax3.set_ylabel('Density', fontsize=FONT_SIZE)
    ax3.set_title(f'Distribution of {measure_name}', fontsize=FONT_SIZE, loc='center')

plt.tight_layout()
plt.show()

SystemExit: /home/milky/drosophila-SNA/data/results/local_measures_snapshots/30_sec_window/ is invalid path!